In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv1D, LSTM, CuDNNLSTM, Flatten, Dropout
from keras import optimizers
from keras.utils import to_categorical

Using TensorFlow backend.


In [3]:
data = pd.read_csv("Drill1Opportunity_taskB2.csv",header=None)
data.rename(columns ={113: 'labels'}, inplace =True)

In [4]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
one_hot_labels = encoder.fit_transform(data['labels'].values.reshape(-1,1)).toarray()
one_hot_labels.shape

(54915, 18)

In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data.drop(['labels'], 1, inplace=True)
data.shape
data = scaler.fit_transform(data)

In [7]:
class trainerHelper():
    
    def __init__(self, data, labels, win_size, num_sensor_data, num_classes):
        self.data = data
        self.labels = labels
        self.win_size = win_size
        self.sensor_data = num_sensor_data
        self.num_classes = num_classes
        self.data_length = self.data.shape[0]
        print(self.data_length)
        self.start = 0
        
    def windows(self):
 
        while self.start + self.win_size < self.data_length:
            yield int(self.start), int(self.start + self.win_size)
            self.start += (self.win_size/2)
    
    def segment_data(self):
        self.start = 0
        segments = np.empty((0, self.win_size, self.sensor_data))
        labels= np.empty((0, self.num_classes))
        for (start, end) in self.windows():
            x = np.zeros((1, self.win_size, self.sensor_data))
            x[0,:] = self.data[start:end,:]
            label = self.labels[start:end,:]
            if(x.shape[1] == self.win_size):
                segments = np.vstack([segments,x])
                lb = np.zeros((1, self.num_classes))
                lb[0,:] = label[-1]
                labels = np.vstack([labels,lb])
        return segments, labels

In [8]:
trainer_helper = trainerHelper(data, one_hot_labels, 15, 113, 18)

54915


In [9]:
segments, labels = trainer_helper.segment_data()

In [10]:
segments.shape

(7320, 15, 113)

In [11]:
labels.shape

(7320, 18)

In [12]:
tot_segments = segments
tot_labels = labels

In [13]:
# maybe we need to normalize the data
num_rows = segments.shape[1]
num_cols = segments.shape[2]
trainSplitRatio = 0.8
# reshaping segments for network input
reshapedSegments = segments.transpose(0,2,1)
# splitting segments in training and testing data
trainSplit = np.random.rand(len(reshapedSegments)) < trainSplitRatio
trainX = reshapedSegments[trainSplit]
testX = reshapedSegments[~trainSplit]
trainY = labels[trainSplit]
testY = labels[~trainSplit]

In [14]:
trainX.shape

(5847, 113, 15)

In [15]:
print(reshapedSegments[0][0])

[ 0.56200975  0.55776977  0.57197371  0.56794573  0.57070172  0.56476574
  0.55416578  0.55904176  0.55882976  0.55522578  0.55819377  0.54526182
  0.56010176  0.55458978  0.54610982]


In [16]:
model = Sequential() #permettere di usare la struttura a stack

In [18]:
win_size = 15
classes = 18
num_sensors = 113
kernel_height = 5
inputshape = (num_sensors, win_size)

In [19]:
model.add(Conv1D(64, kernel_size=kernel_height, strides=1,
                 activation='relu',
                 input_shape=(num_sensors,15)))

In [20]:
model.add(Conv1D(64, kernel_size=kernel_height, strides=1,
                 activation='relu'))

In [21]:
model.add(Conv1D(64, kernel_size=kernel_height, strides=1,
                 activation='relu'))

In [22]:
model.add(Conv1D(64, kernel_size=kernel_height, strides=1,
                 activation='relu'))

In [23]:
model.add(Dropout(0.5))

In [24]:
model.add(CuDNNLSTM(128, return_sequences=True))

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [25]:
model.add(Dropout(0.5))

In [26]:
model.add(CuDNNLSTM(128, return_sequences=False))

In [27]:
model.add(Dropout(0.5))

In [28]:
model.add(Dense(classes, activation='softmax'))

In [29]:
adam = optimizers.RMSprop(lr=0.001, decay=1e-6) 
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [30]:
for layer in model.layers:
    print(layer.name)

conv1d_1
conv1d_2
conv1d_3
conv1d_4
dropout_1
cu_dnnlstm_1
dropout_2
cu_dnnlstm_2
dropout_3
dense_1


In [31]:
import keras as keras

In [32]:
batchSize = 100
keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
model.fit(trainX,trainY, validation_split=1-trainSplitRatio,epochs=100,batch_size=batchSize,verbose=1)
score = model.evaluate(testX,testY,verbose=1)
model.save('tommyCUDACNNmodel.h5')

Train on 4677 samples, validate on 1170 samples
Epoch 1/100
4677/4677 [==============================] - 9s 2ms/step - loss: 2.4681 - acc: 0.3477 - val_loss: 2.2060 - val_acc: 0.4274
Epoch 2/100
4677/4677 [==============================] - 2s 490us/step - loss: 2.4017 - acc: 0.3560 - val_loss: 2.1401 - val_acc: 0.4274
Epoch 3/100
4677/4677 [==============================] - 2s 490us/step - loss: 2.3995 - acc: 0.3556 - val_loss: 2.1934 - val_acc: 0.4274
Epoch 4/100
4677/4677 [==============================] - 2s 488us/step - loss: 2.2806 - acc: 0.3583 - val_loss: 1.8554 - val_acc: 0.4325
Epoch 5/100
4677/4677 [==============================] - 2s 487us/step - loss: 2.0636 - acc: 0.3671 - val_loss: 1.7002 - val_acc: 0.4615
Epoch 6/100
4677/4677 [==============================] - 2s 492us/step - loss: 1.8421 - acc: 0.3928 - val_loss: 1.4921 - val_acc: 0.4897
Epoch 7/100
4677/4677 [==============================] - 2s 486us/step - loss: 1.7011 - acc: 0.4090 - val_loss: 1.8987 - val_acc: 0.

4677/4677 [==============================] - 2s 486us/step - loss: 0.2905 - acc: 0.8948 - val_loss: 0.4072 - val_acc: 0.8795
Epoch 61/100
4677/4677 [==============================] - 2s 489us/step - loss: 0.3002 - acc: 0.8905 - val_loss: 0.4094 - val_acc: 0.8744
Epoch 62/100
4677/4677 [==============================] - 2s 487us/step - loss: 0.2851 - acc: 0.8997 - val_loss: 0.3512 - val_acc: 0.8786
Epoch 63/100
4677/4677 [==============================] - 2s 487us/step - loss: 0.2599 - acc: 0.9087 - val_loss: 0.5543 - val_acc: 0.8316
Epoch 64/100
4677/4677 [==============================] - 2s 487us/step - loss: 0.2835 - acc: 0.9012 - val_loss: 0.4137 - val_acc: 0.8889
Epoch 65/100
4677/4677 [==============================] - 2s 484us/step - loss: 0.2581 - acc: 0.9102 - val_loss: 0.4030 - val_acc: 0.8838
Epoch 66/100
4677/4677 [==============================] - 2s 486us/step - loss: 0.2579 - acc: 0.9089 - val_loss: 0.4896 - val_acc: 0.8581
Epoch 67/100
4677/4677 [=======================

In [33]:
print(score)

[0.38436227645924964, 0.89816700610997968]


In [42]:
predictions = model.predict(testX)
predictions.shape

(1496, 18)

In [32]:
count = 0.0
den = 0.0
for pair in zip(predictions, testY):
    if np.argmax(pair[0]) == np.argmax(pair[1]):
        count += 1.0
    den += 1.0

print(count / den)

0.9211229946524064


In [38]:
pred_df = pd.DataFrame(predictions)
pred_df.to_csv('preds_test.csv', header=False, index=False)

In [39]:
true_df = pd.DataFrame(testY)
true_df.to_csv('true_test.csv', header=False, index=False)